In [12]:
import json
import os
import pandas as pd
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import clear_output

In [13]:
client_id = "00643d1b803a42e394b3ad5ab1d25301"  
client_secret = "8fd4f17a06f342a3a7f639b9f8e02dfe"

In [14]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
def get_track_feature(track_id_array):
    return sp.audio_features(track_id_array)

In [15]:

filter_data = pd.read_csv("../../../handle/song_data.csv")
update_data = filter_data[filter_data['danceability'].isnull()]
link_arr = list(update_data["spotify_link"])
parts = [link_arr[i:i+50] for i in range(0, len(link_arr), 50)]

C:\Users\DELL\AppData\Local\Temp\ipykernel_23056\3452499051.py:1: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  filter_data = pd.read_csv("../../../handle/song_data.csv")


In [10]:
fetch_data = []
for id, part in enumerate(parts):
###### 1:
    if id <= 3500:
        continue 
########
    try:
        print(id)
        fetch_data.append(get_track_feature(part))
        clear_output(wait=True)
    except Exception as e:
        print(f"An error occurred: {e}")  
        break
fetch_data = [x for xs in fetch_data for x in xs]

1999


Max Retries reached


An error occurred: http status: 429, code:-1 - /v1/audio-features/?ids=60oIQLIzRDy7jXASNX5Ix6,7o4vqu8A6Y01XoX4F2D7zo,3fkxYKo0Vq3uynItWEHogm,1nUI5XCjwiVc2kefbz8L3C,4Viub0m9e8EI0415ICCZB0,0bMJ0h8OCsHXEXtqmnJlm0,0NgMPHAl54UN9XrKGH6sFJ,0RfNss0pX8UmaPWr5WsKM2,0RfNss0pX8UmaPWr5WsKM2,4JXwo0Q6XQPKtC4ACbPi8t,2lY8Q0LB4ds8yQQ54fmHdT,6iWSlkckwJyzGDkEX8pNqg,4Pe3LGd5qy3KwmKCbiIsKR,17W5uFdictRAx0pVVxaQe6,6Np34BSo4JE7yl7tABbYoM,0oaRm4BkQyCgyhsMSVilYz,4gdqihSZKMs4l72HXNmwXY,16uTcNIZ0zKgJDQ9dX1Ke9,7xktiByYleckF9Awi7D3B9,2SH6wha9tgm3FaynSq4kne,7hEWsWajkW0qb2oYHMmPzj,0fJ6eJwLmpuPfhaZJwrYS2,3xtvd7QhQf4N968a29e1O0,3xtvd7QhQf4N968a29e1O0,6435ttqDigWL7w4zirJC84,10IpQuIIaKlGr00sw6oZ9r,6j3fuoeDBNV0H8N85u6O91,1h8SFz131kdIxvxxDW3M39,2JKgHnXIfjZFv95yHOXO2e,72Q27eXbIzehy1Q4uN1Ke8,2m0WHvTdoN0rPs2P5wK7OC,3LctKqWFLjftXoIlUb5MLg,4DHY7CjL5BDKmP2QJhZrTC,2aLyq4BFig5FadA8EwtkFy,0L4vXxz212JRxQbJmmr0gu,02SHhIbDoIK6V1h4OiYld8,4EcyN2nZTxjroigeSKOPzb,58r0EiBBjax5J0U2w2B7fS,3wOvcsL8tSDltGBTqz3xJn,0MiBAqwNs44h3Ps3zv7Jzt,1X7A5quOk

In [11]:
for id, feature_dict in enumerate(fetch_data):
    try:
        print(f'{id}/{len(fetch_data)}')
        track_id = feature_dict['id']
        matching_row = update_data[update_data['spotify_link'] == track_id]
        if not matching_row.empty:
            common_features = set(update_data.columns) & set(feature_dict.keys())
            for feature in common_features:
                if feature != 'id':
                    update_data.loc[matching_row.index, feature] = feature_dict[feature]
    except TypeError as e:
        print(f"Caught a TypeError: {e}") 
        continue
    except Exception as e:
        print(f"An error occurred: {e}")
        continue
    clear_output(wait=True)
update_data.to_csv('../../../handle/song_data3.csv', index=False)